In [1]:
import os
import re

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
from langchain_google_genai import GoogleGenerativeAI

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)

OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4"

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", None)

/Users/la-cyrusyc/miniconda3/envs/llamp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
top_llm = GoogleGenerativeAI(
    temperature=0.1,
    model="models/gemini-pro",
    google_api_key=GOOGLE_API_KEY,
    # streaming=True,
    # callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = GoogleGenerativeAI(
    temperature=0.1,
    model="models/gemini-pro",
    google_api_key=GOOGLE_API_KEY,
    max_retries=5,
    # streaming=True,
    # callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool()]
tools += [
    MPThermoExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    # ArxivAgent(llm=bottom_llm).as_tool(agent_kwargs=dict(return_intermediate_steps=False)),
    arxiv,
    wikipedia,
]

instructions = re.sub(
        r"\s+",
        " ",
        """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, Wikipedia, and a python 
    REPL, which you can use to execute python code. If you get an error, debug 
    your code and try again. Only use the output of your code to answer the 
    question. Ask user to clarify their queries if needed. Please note that you 
    don't have direct control over MP but through multiple assistant agents to 
    help you. You need to provide complete context in the input for assistants to 
    do their job.
    """,
    ).replace("\n", " ")
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)


In [5]:
agent_executor.invoke(
    {
        "input": "Please give me the mass density (in g/cm^3) and volume per atom (in Angstrom^3) of stable NaCl.",
    }
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: MPStructureRetriever
Action Input: Retrieve stable NaCl structure in Materials Project.

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "chemsys": "NaCl",
    "is_stable": true,
    "limit": 1
  }
}
```Error on search_materials_summary__get: `fields` must be specified in the query. Please provide more detailed request arguments or try smaller request by specifying 'limit' in request.Action:
```
{
  "action": "Final Answer",
  "action_input": "Please provide more detailed request arguments or try smaller request by specifying 'limit' in request."
}
```

> Finished chain.
{'input': 'Retrieve stable NaCl structure in Materials Project.', 'output': "Please provide more detailed request arguments or try smaller request by specifying 'limit' in request."}Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: Retrieve NaCl 

Retrieving SummaryDoc documents: 100%|██████████| 3/3 [00:00<00:00, 24769.51it/s]


[{'formula_pretty': 'NaCl', 'volume': 42.955532869553466, 'density': 2.259231700010318, 'material_id': 'mp-22851'}, {'formula_pretty': 'NaCl', 'volume': 284.0823192964595, 'density': 2.0496840871350877, 'material_id': 'mp-1120767'}, {'formula_pretty': 'NaCl', 'volume': 43.625325465502776, 'density': 2.224545043829438, 'material_id': 'mp-22862'}]Action:
```json
{
  "action": "Final Answer",
  "action_input": {
    "volume": 42.955532869553466,
    "density": 2.259231700010318
  }
}
```

> Finished chain.
{'input': "Retrieve NaCl minimum energy entry's summary from Materials Project and extract mass density and volume per atom from it.", 'output': {'volume': 42.955532869553466, 'density': 2.259231700010318}}Do I need to use a tool? No
Final Answer: {"Mass density":2.259231700010318,"Volume per atom":42.955532869553466 }

> Finished chain.


{'input': 'Please give me the mass density (in g/cm^3) and volume per atom (in Angstrom^3) of stable NaCl.',
 'chat_history': [],
 'output': '{"Mass density":2.259231700010318,"Volume per atom":42.955532869553466 }'}

In [6]:

llm_gemini = GoogleGenerativeAI(
    temperature=0,
    model="models/gemini-pro",
    google_api_key=GOOGLE_API_KEY,
    # streaming=True,
    # callbacks=[StreamingStdOutCallbackHandler()],
)

llm_gemini.invoke(agent_executor.memory.chat_memory.messages[0].content)


'**Mass density:** 2.165 g/cm^3\n\n**Volume per atom:** 23.9 Angstrom^3'